In [9]:
import numpy as np  # linear algebra
import pandas as pd  # CSV file
import scipy.io.wavfile as sci_wav  # Open wav files
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy.io import wavfile
import os
import config
import wav2Cep


* Load The WAV files!!!!!!!!!

In [10]:
#Change it to your root directory where you downloaded the dataset
#protip: (make a config.py file and add the path in your machine to it. Name the variable as ROOT_DIR )

ROOT_DIR = config.ROOT_DIR 

# List all files and folders in the directory
all_entries = os.listdir(ROOT_DIR)

# If you want to list only files (excluding directories)
files = [f for f in all_entries if os.path.isfile(os.path.join(ROOT_DIR, f))]

# Print the result
# print("All entries:", all_entries)
# print("Files only:", files)

cats = [f for f in files if f.startswith('cat')]
dogs = [f for f in files if f.startswith('dog')]

print(f"cats path: {cats}")
print(f"dogs path: {dogs}")

cats path: ['cat_1.wav', 'cat_10.wav', 'cat_100.wav', 'cat_101.wav', 'cat_102.wav', 'cat_103.wav', 'cat_105.wav', 'cat_106.wav', 'cat_107.wav', 'cat_108.wav', 'cat_109.wav', 'cat_11.wav', 'cat_110.wav', 'cat_112.wav', 'cat_113.wav', 'cat_114.wav', 'cat_115.wav', 'cat_116.wav', 'cat_117.wav', 'cat_118.wav', 'cat_119.wav', 'cat_12.wav', 'cat_120.wav', 'cat_121.wav', 'cat_122.wav', 'cat_123.wav', 'cat_124.wav', 'cat_125.wav', 'cat_126.wav', 'cat_127.wav', 'cat_128.wav', 'cat_129.wav', 'cat_13.wav', 'cat_130.wav', 'cat_131.wav', 'cat_132.wav', 'cat_133.wav', 'cat_134.wav', 'cat_135.wav', 'cat_136.wav', 'cat_137.wav', 'cat_138.wav', 'cat_139.wav', 'cat_14.wav', 'cat_140.wav', 'cat_141.wav', 'cat_142.wav', 'cat_143.wav', 'cat_144.wav', 'cat_146.wav', 'cat_147.wav', 'cat_148.wav', 'cat_149.wav', 'cat_15.wav', 'cat_150.wav', 'cat_151.wav', 'cat_152.wav', 'cat_153.wav', 'cat_154.wav', 'cat_155.wav', 'cat_156.wav', 'cat_157.wav', 'cat_158.wav', 'cat_159.wav', 'cat_16.wav', 'cat_160.wav', 'cat_16

In [11]:
print(f"{len(cats)=} files")
print(f"{len(dogs)=} files")

len(cats)=164 files
len(dogs)=113 files


* Check the total length of samples.

In [12]:
def Length_of_wavs(wavs: list, root: str) -> tuple:
    """
    Check if the durations of cats and dogs audio
    are comparable to avoid an unbalanced dataset.
    
    Also checks the sample rate of each file to ensure there are no outliers.
    
    Now includes minimum duration across all files.
    """
    duration = 0
    rates = []
    min_duration = float('inf')

    for i in wavs:
        rate, data = wavfile.read(root + i)
        file_duration = len(data) / float(rate)
        rates.append(rate)
        duration += file_duration
        if file_duration < min_duration:
            min_duration = file_duration

    avg_duration = duration / len(wavs)
    
    return (duration, set(rates), avg_duration, min_duration)
             #The set() function is used to check if the values in the list are unique

duration_cats, Sampleratescats, avgDurationCat, min_durationCat = Length_of_wavs(cats,ROOT_DIR)
duration_dogs, Sampleratesdogs, avgDurationDog, min_durationDog = Length_of_wavs(dogs,ROOT_DIR)
print(f"Duration of cats : {duration_cats:.2f} seconds, min duration: {min_durationCat:.2f} and avg duration : {avgDurationCat:.2f} seconds")
print(f"Duration of dogs : {duration_dogs:.2f} seconds, min duration: {min_durationDog:.2f} and avg duration : {avgDurationDog:.2f} seconds")
print(f"Unique sample rates = {Sampleratescats, Sampleratesdogs}")



Duration of cats : 1323.90 seconds, min duration: 0.92 and avg duration : 8.07 seconds
Duration of dogs : 598.44 seconds, min duration: 1.02 and avg duration : 5.30 seconds
Unique sample rates = ({16000}, {16000})


* Lets try to get speech only parts from the audios (Have to figure this out)

In [ ]:
# from vad import EnergyVAD              #I think this apprach is based on energy density in the signal to detect the speech intervals
# import soundfile as sf

# test_audio = ROOT_DIR + 'cat_25.wav'    #A random exampple
# saveas = config.SavePath  #path wwhere you wanna save the processed file

# #play the example
# Srate, audio = wavfile.read(test_audio)
# sd.play(audio, Srate)
# sd.wait()




In [ ]:
# #Try to eliminate long pauses

# audio, sr = sf.read(test_audio)
# if audio.ndim == 1:
#     audio = np.expand_dims(audio, axis=0)  # Shape becomes (1, num_samples)
# vad = EnergyVAD(sample_rate=sr)
# speech_signal = vad.apply_vad(audio)

# #Try to add pause

# # Define silence duration in seconds
# padding_duration = 1.0 
# padding_samples = int(sr * padding_duration)

# # Create silence
# silence = np.zeros(padding_samples)

# print("Speech signal shape:", speech_signal.shape)
# speech_signal = speech_signal.flatten()
# speech_signal = speech_signal.astype('float32')
# # Add silence to beginning and end
# padded_signal = np.concatenate([silence, speech_signal, silence])

# sf.write(saveas, padded_signal, sr)

In [ ]:
# #TO DO (Looks like the pauses at the begining and end are eliminated.Would be nice to include that as well )

# Srate, audio = wavfile.read(saveas)
# print(Srate)
# sd.play(audio, Srate)
# sd.wait()


* ################################# Get the mel Coeficients ##################################################

In [ ]:
test_audio = ROOT_DIR + 'cat_25.wav' 
Test_Mfcc = wav2Cep.compute_mfcc(file=test_audio, frameSize=0.300, CepCoeficients=20)
print((Test_Mfcc.shape))

In [ ]:
Test_Mfcc = wav2Cep.compute_2D_mfcc(file=test_audio, frameSize=0.300, CepCoeficients=20, max_frames=30)
print((Test_Mfcc.shape))

* ========== Compute MFCC for all audio files and save the "n" CC features with labels =======================

In [ ]:
def Label_and_Save_1D(Mfcc, label, savePath):

    label_cat = 0  
    label_dog = 1
    num_frames = Mfcc.shape[0]

    if label == 'cat':
        labels = np.full((num_frames, 1), label_cat)
        features_with_labels = np.hstack((Mfcc, labels))
    
    elif label == "dog":
        labels = np.full((num_frames, 1), label_dog)
        features_with_labels = np.hstack((Mfcc, labels))


    # csv_test = config.Features
    df = pd.DataFrame(features_with_labels)
    #df.to_csv(Root + "mfcc_features.csv", index=False, header=False)
    df.to_csv(savePath, mode='a', header=False, index=False)



In [ ]:
#Compute MFCC and append to csv

label_dog = 'dog'
label_cat = 'cat'
Root = config.Features
frameSize = 0.300
CepCoeficients = 20
numRandomSamples = 5
savePath = Root + f'Rdata_fs{int(frameSize*1000)}_cc{CepCoeficients}.csv'

if os.path.exists(savePath):
    print("The Csv file already exists")

else:
    
    for i in range(numRandomSamples):
        for dog in dogs:
            audio = ROOT_DIR + dog 
            MFCC = wav2Cep.compute_mfcc(audio, frameSize = frameSize, CepCoeficients = CepCoeficients,random_segment_duration=5)
            Label_and_Save_1D(Mfcc= MFCC, label=label_dog, savePath=savePath)

        for cat in cats:
            audio = ROOT_DIR + cat 
            MFCC = wav2Cep.compute_mfcc(audio, frameSize = frameSize, CepCoeficients = CepCoeficients, random_segment_duration=8)
            Label_and_Save_1D(Mfcc= MFCC, label=label_cat, savePath=savePath)

    
    
    

* Try to stack the MFCCs as 2D array(.npy format) for a CNN network

In [13]:

def add_sample(Mfcc, label, features_list, labels_list):
    label_map = {'cat': 0, 'dog': 1}
    
    if label not in label_map:
        raise ValueError(f"Unknown label: {label}")
    
    label_value = label_map[label]
    
    # Append to your dataset
    features_list.append(Mfcc)
    labels_list.append(label_value)



In [16]:
label_dog = 'dog'
label_cat = 'cat'
Root = config.Features
frameSize = 0.300
CepCoeficients = 20
numRandomSamples = 5
MAXFrames = 50
savePath = Root + f'2Ddata_fs{int(frameSize*1000)}_cc{CepCoeficients}_f{MAXFrames}/'

if os.path.exists(savePath):
    print("The file or folder already exists")

else:
    os.makedirs(savePath, exist_ok=True)
    features_dataset = []
    labels_dataset = []
    for i in range(numRandomSamples):
        for dog in dogs:
            audio = ROOT_DIR + dog 
            MFCC = wav2Cep.compute_2D_mfcc(audio, frameSize = frameSize, CepCoeficients = CepCoeficients,
                                           random_segment_duration=5, max_frames=MAXFrames)
            add_sample(MFCC, label_dog, features_dataset, labels_dataset)
            

        for cat in cats:
            audio = ROOT_DIR + cat 
            MFCC = wav2Cep.compute_2D_mfcc(audio, frameSize = frameSize, CepCoeficients = CepCoeficients, 
                                           random_segment_duration=8, max_frames=MAXFrames)
            add_sample(MFCC, label_cat, features_dataset, labels_dataset)

    np.save(savePath + 'features.npy', np.array(features_dataset))
    np.save(savePath + 'labels.npy', np.array(labels_dataset))


